# dynamo query table

In [33]:
import os
import json
import time
import boto3
import tweepy
import pprint
import datetime
import numpy as np
import pandas as pd
import botocore.session
from dynamodb_json import json_util
from boto3.dynamodb.conditions import Key, Attr
# from tweet_grab

with open("../creds.json", "r") as f:
    creds = json.load(f)
    f.close()

auth = tweepy.OAuthHandler(creds['twitter-api-key'], creds['twitter-secret-key'])
auth.set_access_token(creds['twitter-access-token'], creds['twitter-secret-access'])

api = tweepy.API(auth)

client = boto3.client('dynamodb',
  endpoint_url = "http://localhost:8000",
  aws_access_key_id=creds["aws-access-key"],
  aws_secret_access_key=creds["aws-secret-key"],
  region_name='us-east-2')

dynamodb = boto3.resource('dynamodb', \
            endpoint_url = "http://localhost:8000", \
            region_name='us-east-2')

tweets = dynamodb.Table("tweets")
print(tweets)
    
print("imported modules.")

dynamodb.Table(name='tweets')
imported modules.


## get following

In [15]:
os.getcwd()

'/home/steven/twitter.ai/notebooks'

In [16]:
os.chdir('/home/steven/twitter.ai/main')
import tweet_grab

os.chdir('/home/steven/twitter.ai/notebooks')

imported modules successfully.


In [19]:
try:
    my_following = tweet_grab.get_followers("xanax_princess_", api)

except Exception as e:
    print("Exception: {}".format(e))

got most recent 100 following
got next 100 following
got next 100 following
got next 100 following
got next 100 following


## query for user

In [7]:
user = "missnikkay"
df = tweets.query(KeyConditionExpression=Key('user').eq(user))
df["Count"]

0

## scan table

In [34]:
response = tweets.scan()
response["Count"]

5035

## response items

In [4]:
response["Items"]

[{'favorites': Decimal('0'),
  'created_at': '2022-10-30T17:11:49Z',
  'tweet': 'RT @KALMARKSBAND: Playing with @pilemusic tonight at Crystal Ballroom https://t.co/KiATGl6R38',
  'id': Decimal('1586767843676364802'),
  'retweets': Decimal('1'),
  'user': 'pilemusic',
  'inserted_at': '2023-04-27T12:24:07Z'},
 {'favorites': Decimal('0'),
  'created_at': '2022-10-30T17:11:54Z',
  'tweet': 'RT @EISRecords: EIS LIVE - 10/30:\n\n* @pilemusic and @KALMARKSBAND in Boston at @CrystalSville\n\n* @stuckchi in DC at @pieshopdc w/ Teen Mort…',
  'id': Decimal('1586767863527907329'),
  'retweets': Decimal('5'),
  'user': 'pilemusic',
  'inserted_at': '2023-04-27T12:24:07Z'},
 {'favorites': Decimal('5'),
  'created_at': '2022-10-31T01:27:08Z',
  'tweet': '@gimmetinnitus Hopefully @StephenM',
  'id': Decimal('1586892493844582400'),
  'retweets': Decimal('0'),
  'user': 'pilemusic',
  'inserted_at': '2023-04-27T12:24:07Z'},
 {'favorites': Decimal('0'),
  'created_at': '2022-10-31T17:27:37Z',
  'tweet'

## pandas

In [30]:
df = pd.DataFrame(json_util.loads(response["Items"]))
df.head()

,favorites,created_at,tweet,id,retweets,user,inserted_at
0,29,2023-03-19T17:21:45Z,i feel a lot of art about to come out of me. a...,1637504645252743170,0,NECROMANClNG,2023-04-28T14:13:05Z
1,13,2023-03-19T19:57:08Z,@LauraJaneGrace i was listening to against me ...,1637543750992814081,0,NECROMANClNG,2023-04-28T14:13:05Z
2,2,2023-03-19T20:05:32Z,@LauraJaneGrace thanks 🫶🏻,1637545864565596160,0,NECROMANClNG,2023-04-28T14:13:05Z
3,1935,2023-03-19T20:18:57Z,i love being me because sometimes i get bored ...,1637549239235489794,41,NECROMANClNG,2023-04-28T14:13:05Z
4,53,2023-03-19T20:20:07Z,the kinds of art i do are really satisfying fo...,1637549530894802946,0,NECROMANClNG,2023-04-28T14:13:05Z


## summary stats

In [31]:
df.groupby(['user']).agg({'created_at': ['min', 'max'], 'tweet': ['count']})

created_at                       tweet
                                  min                   max count
user                                                             
Carnage4Life     2023-04-01T09:38:11Z  2023-04-28T16:19:21Z   579
CaucasianJames   2022-08-13T16:54:09Z  2023-04-28T05:14:16Z   597
NECROMANClNG     2023-03-19T17:21:45Z  2023-04-28T14:23:49Z   591
TProphet         2022-11-07T19:03:28Z  2022-11-20T02:17:48Z   586
fixedpointfae    2023-01-27T05:19:08Z  2023-04-25T16:58:35Z   585
javavvitch       2023-04-10T16:17:14Z  2023-04-28T18:53:02Z   540
keribla          2023-03-24T16:58:17Z  2023-04-28T16:08:43Z   599
knifeplay_music  2017-08-05T00:37:23Z  2023-04-26T21:30:43Z   366
pilemusic        2022-10-30T17:11:49Z  2023-04-26T21:15:08Z   592

## unique users

In [32]:
dynamo_users = list(np.unique(df['user']))
dynamo_users

['Carnage4Life',
 'CaucasianJames',
 'NECROMANClNG',
 'TProphet',
 'fixedpointfae',
 'javavvitch',
 'keribla',
 'knifeplay_music',
 'pilemusic']

## find missing users in dynamo

In [20]:
my_following

['bastard_band',
 'datacoves',
 'punished_cait',
 'pdrmnvd',
 'ChaunceyCC',
 'MerrittBaer',
 'jasminericegirl',
 'artisinalsprite',
 'Sahelanth',
 'emilianbold',
 'loige',
 'halsnew',
 'pinkninjajess',
 'Amy_Henderson__',
 'breathingatlast',
 'ladaw89',
 'dankbees7',
 'allisonavani1',
 'YeetToOrbit',
 'CalMeNoble6',
 'somethin_2b',
 'notrealneverwas',
 'pilemusic',
 'krilliamhmacy',
 'jpegmafia',
 'kittywavvy',
 'elena_lape',
 'esevatojotx',
 'rrossjd',
 'kittyxcate',
 'hahnscratch',
 'MischievousMal',
 'casiebones',
 'DigitalBlkHippy',
 'ennntropy',
 'shanley',
 'AnnieisUndead',
 'mollysoda',
 'Aliafonzy43',
 'Carnage4Life',
 'fixedpointfae',
 'bernhardsson',
 'freshhel',
 'kvlly',
 'seanjtaylor',
 'ecto_fun',
 'colborkmundus',
 'dataandme',
 'robinson_es',
 'dniellechelosky',
 'LittleMissData',
 'beeonaposy',
 'JoannaNewSum41',
 'titsay',
 'ZeroSuitCamus',
 'keribla',
 'vboykis',
 'Pdaddynthehyena',
 'coolgirl0nline',
 'endingwithali',
 'JamieMuma',
 'jsedlacekjr',
 'tagabowPHL',
 'y

In [26]:
subset = [a not in dynamo_users for a in my_following]
missing_users = [x for x, y in zip(my_following, subset) if y]
missing_users

['punished_cait',
 'pdrmnvd',
 'MerrittBaer',
 'jasminericegirl',
 'artisinalsprite',
 'Sahelanth',
 'emilianbold',
 'halsnew',
 'pinkninjajess',
 'Amy_Henderson__',
 'breathingatlast',
 'allisonavani1',
 'YeetToOrbit',
 'CalMeNoble6',
 'notrealneverwas',
 'jpegmafia',
 'elena_lape',
 'esevatojotx',
 'rrossjd',
 'kittyxcate',
 'hahnscratch',
 'casiebones',
 'DigitalBlkHippy',
 'ennntropy',
 'shanley',
 'AnnieisUndead',
 'mollysoda',
 'Aliafonzy43',
 'Carnage4Life',
 'fixedpointfae',
 'bernhardsson',
 'freshhel',
 'kvlly',
 'seanjtaylor',
 'ecto_fun',
 'colborkmundus',
 'dataandme',
 'robinson_es',
 'dniellechelosky',
 'LittleMissData',
 'beeonaposy',
 'JoannaNewSum41',
 'titsay',
 'ZeroSuitCamus',
 'keribla',
 'vboykis',
 'Pdaddynthehyena',
 'coolgirl0nline',
 'endingwithali',
 'JamieMuma',
 'jsedlacekjr',
 'tagabowPHL',
 'yrinfinitycrush',
 'osoosoband',
 'sama',
 'youngtiddy',
 'FranziaMom',
 'pilatesdev',
 'mike_julian',
 'knifeplay_music',
 'joneaux',
 'GergelyOrosz',
 'p8stie',
 '